In [61]:
import pandas as pd
import altair as alt
from pyprojroot import here

from src.visualization import visualize as viz

In [62]:
df = pd.read_excel(here('./data/raw/istat/decessi/Tavola riepilogativa_30APRILE_7.270COMUNI.xlsx'), header=1)
df.columns = ['codice_regione', 'codice_provincia', 'regione', 'provincia', 'comune', 'codice_comune', 'jan_1519', 'feb_1519', 'mar_1519', 'apr_1519', 'jan_20', 'feb_20', 'mar_20', 'apr_20', 'jan_incr', 'feb_incr', 'mar_incr', 'apr_incr']

In [83]:
prov = df.groupby(['provincia', 'regione']).sum().iloc[:, 3:].reset_index()

for month in ['jan', 'feb', 'mar', 'apr']:
    prov[f'{month}_incr'] = 100.0 * (prov[f'{month}_20'] / prov[f'{month}_1519'] - 1)

In [84]:
top_apr = prov.sort_values('apr_incr', ascending=False).head(50)
m = pd.melt(top_apr, id_vars=['provincia'], value_vars=['mar_incr', 'apr_incr'])
m['mese'] = m.variable.str.split('_').str[0]

In [85]:
alt.Chart(m).mark_point().encode(
    x = alt.X('value', title='Incremento percentuale numero decessi'),
    y = alt.Y('provincia', axis=alt.Axis(grid=True), sort='-x'),
    #row = alt.Row('provincia'),
    color = 'mese'
).properties(
    width = 500,
    height = 800
)

alt.Chart(...)

In [86]:
alt.Chart(top_apr).mark_bar().encode(
    x = 'apr_incr',
    y = alt.Y('provincia', sort='-x'),
    color='regione:N'
).properties(
    height=800
)

alt.Chart(...)

In [87]:
prov = df.groupby(['provincia', 'regione', 'codice_provincia']).sum().iloc[:, 3:].reset_index()

In [88]:
prov

,provincia,regione,codice_provincia,feb_1519,mar_1519,apr_1519,jan_20,feb_20,mar_20,apr_20
0,Agrigento,Sicilia,84,431.0,442.0,384.8,444,385,421,321
1,Alessandria,Piemonte,6,537.2,530.4,492.4,557,480,1024,900
2,Ancona,Marche,42,412.8,400.0,380.0,424,367,607,451
3,Arezzo,Toscana,51,344.2,361.8,325.8,418,329,366,370
4,Ascoli Piceno,Marche,44,193.4,212.6,181.0,215,188,193,194
...,...,...,...,...,...,...,...,...,...,...
102,Vercelli,Piemonte,2,203.8,197.4,183.2,185,189,382,318
103,Verona,Veneto,23,740.8,755.8,666.6,864,764,1022,966
104,Vibo Valentia,Calabria,102,139.6,150.4,126.2,171,152,151,125
105,Vicenza,Veneto,24,686.2,680.6,612.6,767,663,814,805


In [89]:
import geopandas as gpd
import matplotlib.pyplot as plt
from pyprojroot import here
from src.data.read_dataset import get_processed_dataset
from src.visualization import visualize

prov_df = get_processed_dataset('dpc_province')
italy = gpd.read_file('../../data/raw/prov2019.geojson')